In [72]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
data_input = open('ocr_conf.pkl','rb')
(digit_arr, digt_marsk_arr, empty_arr,char_s_arr,char_b_arr) = pickle.load(data_input)
data_input.close()
digit_width=6
digit_sep=2
colon_width=1
x_axis_start=1
digit_hight=11
row_sep=6
col_sep=317
pixel_threshold=190
first_time_col=50
first_deal_col_bound=192
first_buy_col_bound=252
first_sell_col_bound=307

In [6]:
def ocrTimeArr(coltime):
    y=coltime.shape[1]
    hight=coltime.shape[0]
    second1=coltime[:,y-digit_width:y]
    y=y-digit_width-digit_sep
    second2=coltime[:,y-digit_width:y]
    y=y-digit_width-digit_sep-colon_width-digit_sep
    minute1=coltime[:,y-digit_width:y]
    y=y-digit_width-digit_sep
    minute2=coltime[:,y-digit_width:y]
    y=y-digit_width-digit_sep-colon_width-digit_sep
    hour1=coltime[:,y-digit_width:y]
    return (hour1,minute2,minute1,second2,second1)

In [7]:
def arr2int(arr,last=None):
    if last is not None:
        if np.logical_or(arr ==digit_arr[last] ,digt_marsk_arr[last]).all():
            return 1,last
    
    for (cid,c) in enumerate( digit_arr):
        if np.logical_or(arr ==digit_arr[cid] ,digt_marsk_arr[cid]).all():
            return 1,cid
    return 0,arr

In [8]:
def ocrTimeCol(coltime,lasttime = "start"):
    #     pdb.set_trace()
    x = x_axis_start   # config
    #     total is 57 row ,notice whether has 57 row in each colum section when call from pyautogui,
    last_time_arrs=[9,2,5,0,0]
    printlist=[]
    for i in range(round(coltime.shape[0] / (digit_hight+row_sep)) - 1):
        timearrs = ocrTimeArr(coltime[x:x + digit_hight, :])
        if (timearrs[0] == empty_arr).all():
            # print(lasttime)
            printlist.append(lasttime)
        else:
            outstr = ""
            for (tid,_) in enumerate(timearrs):
                if tid == len(timearrs)-1:
                    ret, ret_val = arr2int(timearrs[tid])
                else:
                    ret, ret_val = arr2int(timearrs[tid],last_time_arrs[tid])
                if not ret:
#                     print(ret_val)
                    return ret_val,lasttime
                last_time_arrs[tid]=ret_val
                outstr = outstr + str(ret_val)

            # print(i, outstr)
            printlist.append(outstr)
            lasttime = outstr
        x = x + digit_hight+row_sep
    for p in printlist:
        print(p)
    return ret_val,lasttime

In [9]:
import pdb
import time
def updateMarsk(id,_arr):
    #pdb.set_trace()

#     print("before update marsk:",id)
#     print(digt_marsk_arr[id])
#     print(np.sum(digt_marsk_arr[id]))
    new=np.logical_or((_arr!=digit_arr[id]),digt_marsk_arr[id])

#     print("after update marsk:",id)
    errsum=np.sum(new)
#     print("after update marsk errsum:",errsum)
    if errsum>20:
        print("not update marsk id {} for error {} >20".format(id,errsum))
        time.sleep(2)
    else:
        digt_marsk_arr[id]=new
    return errsum

In [10]:
def findMatch(arr):
    bestid=0
    besterr=1000
    for (cid,c) in enumerate( digit_arr):
        currenterr=np.sum(arr != c)
        if besterr>currenterr:
            besterr=currenterr
            bestid=cid
    return bestid,besterr

In [11]:
def parselevel2time(arr):
    arrg = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
    arrg[arrg >= pixel_threshold] = 255
    arrg[arrg < pixel_threshold] = 0
    lasttime="start"
    for i in range(round(arrg.shape[1]/col_sep)):
        ret_val, lasttime = ocrTimeCol(arrg[:, :first_time_col+i*col_sep],lasttime)
#         print(ret_val, lasttime)
        while not isinstance(ret_val, int):

            cid, errnum = findMatch(ret_val)
            if errnum > 10:
                print("not findMatch  cid {}, errnum {},ret_val {} !".format(cid, errnum,ret_val))
            if updateMarsk(cid, ret_val) > 10:
                break
            ret_val, lasttime = ocrTimeCol(arrg[:, :first_time_col+i*col_sep],lasttime)

In [62]:
def save_pickle():
    data_output = open('ocr_conf.pkl', 'wb')
    pickle.dump((digit_arr, digt_marsk_arr, empty_arr,char_s_arr,char_b_arr), data_output)
    data_output.close()

In [63]:
save_pickle()

In [13]:
digt_marsk_arr=[empty_arr-1==empty_arr]*10

In [14]:
arr1  =cv2.imread(r"2.jpg")

In [15]:
parselevel2time(arr1)

00959
01002
01005
01005
01007
01008
01008
01008
01008
01008
01008
01008
01010
01010
01010
01017
01018
01021
01021
01024
01024
01024
01027
01028
01028
01029
01029
01043
01047
01054
01056
01059
01059
01100
01100
01102
01110
01112
01112
01117
01117
01117
01117
01117
01117
01117
01119
01119
01119
01119
01119
01119
01119
01120
01122
01122
01122
01227
01124
01125
01129
01130
01130
01130
01136
01138
01138
01138
01138
01138
01138
01138
01138
01138
01138
01141
01141
01146
01146
01150
01151
01151
01151
01158
01158
01201
01202
01202
01202
01202
01202
01202
01202
01202
01202
01211
01211
01215
01215
01218
01218
01219
01219
01219
01219
01219
01224
01224
01227
01227
01227
01235
01235
01235
01355
01355
01355
01237
01237
01237
01237
01237
01238
01242
01242
01242
01242
01242
01243
01243
01243
01244
01244
01247
01247
01256
01256
01309
01309
01313
01313
01313
01313
01313
01313
01313
01313
01313
01315
01320
01346
01349
01354
01355
01357
01402
01412
01422
01437
01437
01437
01440
01442
01442
01442
01442
0144

In [19]:
myarr=np.array([1,2])

In [20]:
def chmyarray(myarr):
    myarr[0]+=2

In [21]:
chmyarray(myarr.copy())

In [22]:
arr1  =cv2.imread(r"2.jpg")


In [23]:
dot_width=1
precision=3

In [24]:
def ocrPriceArr(price_arr,len=5):
    digit_arr=[]
    y=price_arr.shape[1]
    #left of dot
    for i in range(precision):
        digit_arr.append(price_arr[:, y - digit_width:y])
        y = y - digit_width - digit_sep
    y=y-dot_width-digit_sep
    #right of dot
    for i in range(len-precision):
        digit_arr.append(price_arr[:, y - digit_width:y])
        y = y - digit_width - digit_sep
    return digit_arr

In [25]:
def ocrPriceCol(colprice,last_price_str):
    x = x_axis_start
    printlist = []
    last_len=len(last_price_str)
    for i in range(round(colprice.shape[0] / (digit_hight + row_sep)) - 1):
        price_arrs = ocrPriceArr(colprice[x:x + digit_hight, :],last_len+1)#read additional empty dight
        if (price_arrs[last_len] != empty_arr).all():
            last_len+=1  #price increase to more digits
        elif (price_arrs[last_len-2] == empty_arr).all():
            last_len -= 1  #price decrease to less digits

        outstr = ""
        for i in range(last_len):
            if last_len == len(last_price_str):
                ret, ret_val = arr2int(price_arrs[i], int(last_price_str[last_len-i-1]))
            else:
                ret, ret_val = arr2int(price_arrs[i])
            if not ret:
                print(ret_val)
                return ret_val, last_price_str
            outstr += str(ret_val)

        # print(i, outstr)
        printlist.append(outstr[::-1])
        last_price_str = outstr
        x = x + digit_hight + row_sep
    for p in printlist:
        print(p)
    return ret_val, last_price_str

In [44]:
import pdb
price_pixel_threshold=210
def parselevel2price(arr):
    arrg = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
    arrg[arrg >= price_pixel_threshold] = 255
    arrg[arrg < price_pixel_threshold] = 0
    lastprice="142000"
    for i in range(round(arrg.shape[1]/col_sep)):
        ret_val, lastprice = ocrPriceCol(arrg[:, :first_price_col_bound + i * col_sep], lastprice)
        time.sleep(1)
        while not isinstance(ret_val, int):

            cid, errnum = findMatch(ret_val)
            if errnum > 10:
                print("not findMatch  cid {}, errnum {},ret_val {} !".format(cid, errnum,ret_val))
            if updateMarsk(cid, ret_val) > 20:
                break
            ret_val, lastprice = ocrPriceCol(arrg[:, :first_price_col_bound + i * col_sep], lastprice)

In [45]:
path=r"E:\pycharm\kzz\screen_shot\000002"

# arr1  =cv2.imread(os.path.join(path,"304.jpg"))
arr1  =cv2.imread(r"2.jpg")
first_price_col_bound=128

In [41]:
precision=3
parselevel2price(arr1)

[[255 255   0   0   0   0]
 [255   0   0 255 255 255]
 [  0   0 255 255 255 255]
 [  0   0 255 255 255 255]
 [  0   0   0   0   0   0]
 [  0   0 255 255   0   0]
 [  0   0 255 255 255   0]
 [  0   0 255 255 255   0]
 [  0   0 255 255 255   0]
 [  0   0 255 255   0   0]
 [255   0   0   0   0 255]]
[[255   0   0   0   0 255]
 [  0   0 255 255   0   0]
 [255 255 255 255   0   0]
 [255 255 255 255   0   0]
 [255 255 255 255   0   0]
 [255 255 255   0   0   0]
 [255 255   0   0   0 255]
 [255   0   0 255 255 255]
 [  0   0 255 255 255 255]
 [  0   0 255 255 255 255]
 [  0   0   0   0   0   0]]
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648
2648

In [49]:
%matplotlib qt 
plt.figure(figsize=(10, 10))
arrg = cv2.cvtColor(arr1, cv2.COLOR_BGR2GRAY)
plt.imshow(arrg[:,:500])
plt.show()

In [50]:
first_deal_col_bond=192

In [51]:
arrg[arrg >= price_pixel_threshold] = 255
arrg[arrg < price_pixel_threshold] = 0

In [70]:
plt.imshow(arrg[18:29,307-6:307])

In [59]:
char_b_arr=arrg[18:29,192-6:192].copy()

In [58]:
char_s_arr=char_b_arr.copy()

In [67]:
digit_arr[4]

array([[255, 255, 255,   0,   0,   0],
       [255, 255, 255,   0,   0,   0],
       [255, 255,   0,   0,   0,   0],
       [255, 255,   0, 255,   0,   0],
       [255,   0,   0, 255,   0,   0],
       [  0,   0, 255, 255,   0,   0],
       [  0, 255, 255, 255,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255,   0,   0],
       [255, 255, 255, 255,   0,   0],
       [255, 255, 255, 255,   0,   0]], dtype=uint8)

In [108]:
import logging
logger1 = logging.getLogger('root')
FORMAT = "[ qqq:%(lineno)s - %(funcName)20s() ] %(message)s"
logging.basicConfig(format=FORMAT)
logger1.setLevel(logging.DEBUG)

In [102]:
def logf():
    logger1.error(' message')
    return 1,"a"

In [103]:
logf()

[<ipython-input-102-97da9b31173f>:2 -                 logf() ]  message


(1, 'a')

In [78]:
"sdf".find("e")

-1

In [80]:
b=logf()

[<ipython-input-79-14ec3494e226>:2 -                 logf() ] your message


In [81]:
b

(1, 'a')

In [82]:
i=1

In [83]:
i+=1

In [84]:
i

2

In [85]:
""[::-1]

''

In [115]:
 logger = logging.getLogger()
 logger.setLevel(logging.DEBUG)
 
 rht = logging.handlers.TimedRotatingFileHandler("reindex_out.log", 'D')
 fmt = logging.Formatter("%(asctime)s %(pathname)s %(filename)s %(funcName)s %(lineno)s \
      %(levelname)s - %(message)s", "%Y-%m-%d %H:%M:%S")
 rht.setFormatter(fmt)
 logger.addHandler(rht)
 
 debug = logger.debug
 info = logger.info
 warning = logger.warn
 error = logger.error
 critical = logger.critical



In [110]:
logger2 = logging.getLogger('root')
FORMAT = "[ qqq:%(lineno)s - %(funcName)20s() ] %(message)s"
logging.basicConfig(format=FORMAT)
logger2.setLevel(logging.DEBUG)
logger2.error(' message')

[<ipython-input-110-1f7e07e9ec70>:5 -             <module>() ]  message


In [119]:
critical("sdf")

[<ipython-input-119-72c07cc49c4e>:1 -             <module>() ] sdf


In [118]:
logging.shutdown()